<a href="https://colab.research.google.com/github/felladib/H_SentimentAnalysis_REC/blob/main/testNHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import tensorflow as tf
import math
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
import re
import os

from tqdm import tqdm



import keras
from keras.layers import Embedding
from keras.layers.core import Flatten
from keras.layers import Multiply, Dense, Input , Attention
from keras.models import Model
from keras.layers import concatenate
from keras.models import load_model
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import math
from time import time
import pickle
import tqdm

from tensorflow.keras.datasets import imdb
from tensorflow.keras.initializers import lecun_uniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import layers

import matplotlib.pyplot as plt

# **load data set**

In [ ]:
# yelp shop dataset :
dataset = pd.read_csv('/content/drive/MyDrive/yelp_normalized_dataset(55738).csv')

In [ ]:
dataset

In [ ]:
# yelp beauty and spa data set :
# dataset =pd.read_csv('/content/drive/MyDrive/Yelp_normalized_Beauty&Spas_Dataset(76449).csv')

# **Data_cleaning and sentiment analysis**
*cette partie est utilise pour la partie d'analyse de sentiment pour preprocesing de text*

In [ ]:
!pip install flair

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from string import punctuation
from flair.data import Sentence
import pickle
nltk.download('stopwords')

with  open('/content/drive/MyDrive/IMDB_vocab',"rb") as file:
      tokens = pickle.load(file)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def clean_doc(doc):#fonction pour nettoyer les commentaires
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = [word.lower() for word in tokens ]
    return tokens

In [ ]:
#nettoyer les reviews dans le dataset
dataset['text']=dataset['text'].apply(clean_doc)

In [ ]:
reviews = dataset['text']
token = Tokenizer()
token.word_index = tokens

pad=token.texts_to_sequences(reviews)
reviews = pad_sequences(pad, maxlen=1480, padding='post')

In [ ]:
loaded_model_SA = tf.keras.models.load_model('/content/drive/MyDrive/concat_modelCASA')#model de sentiment sans la partie de confiance

In [ ]:
dataset

In [ ]:
column_vector = dataset['valeur_conf'].values.reshape(-1, 1)
confiance = np.tile(column_vector, (1, 5))
print(confiance.shape)

(76449, 5)


In [ ]:
reviews.shape

(76449, 1480)

In [ ]:
sentiment = loaded_model_SA.predict([reviews,confiance])
dataset['SAFR']= sentiment

2390/2390 [==============================] - 124s 52ms/step


In [ ]:
# dataset yelp beauty and spa
dataset.to_csv('/content/drive/MyDrive/Yelp_normalized_Beauty&Spas_Dataset(76449).csv',index=False)

In [ ]:
# dataset yelp shop
dataset.to_csv('/content/drive/MyDrive/yelp_normalized_dataset(55738).csv' ,index=False)

# **Fakereviews**
*la partie d'utilisation de la confiance avec l'analyse de sentiment*

In [ ]:
# relu fonction permet de retourner x or 0 dans le cas ou x<0
def relu(x):
    return (np.maximum(0,x))

#  build_confidence_M fonction permet de retourner un vecteur qui contient la valeur de confiance pour chaque row (user , item ) dans notre dataset
def build_confidence_M(dataset_item, dataset_user, emb_dataset):
  # Group the user and item ratings by user_id and item_id respectively
  user_ratings = dataset_user.groupby('user_id').apply(lambda x: dict(zip(x['item_id'], x['rating'])))
  item_ratings = dataset_item.groupby('item_id').apply(lambda x: dict(zip(x['user_id'], x['rating'])))
  print(user_ratings)
  confidence_M = np.zeros(emb_dataset.shape[0]) # initialize the matrix with zeros

  for i, row in emb_dataset.iterrows():
    print('tuple : ' , i)
    user   = row['user_id']
    item   = row['item_id']
    rating = row['rating']

    # recupere le user qu'on veut lui calculer la valeur de confiance par rapport à un item
    if rating <= 3:
      # Compute the first part of the confidence value using user ratings
      user_ratings_for_item = user_ratings.get(user, {})
      filtered_ratings = [r for u, r in user_ratings_for_item.items() if u != item and r <= 3]
      if filtered_ratings:
        mean1 = sum(filtered_ratings) / len(filtered_ratings)
        result1 = -relu(abs(mean1 - rating) - 0.8)
      else:
        result1 = 0

      # Compute the second part of the confidence value using item ratings
      item_ratings_for_user = item_ratings.get(item, {})
      filtered_ratings = [r for u, r in item_ratings_for_user.items() if u != user and r <= 3]
      if filtered_ratings:
        mean2 = sum(filtered_ratings) / len(filtered_ratings)
        result2 = -relu(abs(mean2 - rating) - 0.8)
      else:
        result2 = 0

    # les meme etap si rating >3
    elif rating > 3:
      # Compute the first part of the confidence value using user ratings
      user_ratings_for_item = user_ratings.get(user, {})
      filtered_ratings = [r for u, r in user_ratings_for_item.items() if u != item and r > 3]
      if filtered_ratings:
        mean1 = sum(filtered_ratings) / len(filtered_ratings)
        result1 = -relu(abs(mean1 - rating) - 0.8)
      else:
        result1 = 0

      # Compute the second part of the confidence value using item ratings
      item_ratings_for_user = item_ratings.get(item, {})
      filtered_ratings = [r for u, r in item_ratings_for_user.items() if u != user and r > 3]
      if filtered_ratings:
        mean2 = sum(filtered_ratings) / len(filtered_ratings)
        result2 = -relu(abs(mean2 - rating) - 0.8)
      else:
        result2 = 0

    result = math.exp(result1 + result2)
    # remplire la matrice par la valeur de confiance
    confidence_M[i] = result

  return confidence_M

In [ ]:
dataset_user = dataset.copy() #copie notre dataset dans dataset1
dataset_item= dataset.sort_values(by=['item_id'],axis=0, ascending = True) #tri dataset1 by items sachant que dataset est trie par users
dataset_emb= dataset.copy() # copie dataset dans dataset2

In [ ]:
matrice_de_confiance = build_confidence_M(dataset_item,dataset_user,dataset_emb)

Streaming output truncated to the last 5000 lines.
tuple :  50738
tuple :  50739
tuple :  50740
tuple :  50741
tuple :  50742
tuple :  50743
tuple :  50744
tuple :  50745
tuple :  50746
tuple :  50747
tuple :  50748
tuple :  50749
tuple :  50750
tuple :  50751
tuple :  50752
tuple :  50753
tuple :  50754
tuple :  50755
tuple :  50756
tuple :  50757
tuple :  50758
tuple :  50759
tuple :  50760
tuple :  50761
tuple :  50762
tuple :  50763
tuple :  50764
tuple :  50765
tuple :  50766
tuple :  50767
tuple :  50768
tuple :  50769
tuple :  50770
tuple :  50771
tuple :  50772
tuple :  50773
tuple :  50774
tuple :  50775
tuple :  50776
tuple :  50777
tuple :  50778
tuple :  50779
tuple :  50780
tuple :  50781
tuple :  50782
tuple :  50783
tuple :  50784
tuple :  50785
tuple :  50786
tuple :  50787
tuple :  50788
tuple :  50789
tuple :  50790
tuple :  50791
tuple :  50792
tuple :  50793
tuple :  50794
tuple :  50795
tuple :  50796
tuple :  50797
tuple :  50798
tuple :  50799
tuple :  50800
tupl

In [ ]:
import numpy as np
values , counts = np.unique( matrice_de_confiance , return_counts = True) #verification que la matrice a ete bien rempli
print(values , counts)

[0.09071795 0.09525293 0.10025884 0.10464906 0.10580539 0.1071706
 0.1078598  0.10942674 0.11080316 0.11329301 0.11360815 0.11426548
 0.11532512 0.11648416 0.1169548  0.11916184 0.12071949 0.1244053
 0.1266071  0.12824072 0.1287349  0.12911409 0.12965773 0.1305024
 0.13089846 0.13199384 0.13278186 0.13326914 0.13533528 0.13693687
 0.13821465 0.13925777 0.13983282 0.14012542 0.14148585 0.14274911
 0.1439252  0.14639767 0.14701195 0.14717557 0.14956862 0.15208232
 0.15259011 0.15317251 0.15335497 0.15355688 0.15403303 0.15562322
 0.15611805 0.15652407 0.15661772 0.15693508 0.15811314 0.15811314
 0.15921497 0.15994897 0.16064289 0.16064289 0.16121764 0.16152798
 0.16185529 0.16256674 0.16321313 0.16380298 0.16529889 0.16714578
 0.16901332 0.16948345 0.16977659 0.16989936 0.17090171 0.17142038
 0.17199427 0.17253712 0.17377394 0.17444359 0.17669445 0.17669445
 0.17669445 0.17713784 0.17738873 0.17767881 0.17881052 0.17939202
 0.18128366 0.18140602 0.18167143 0.18255582 0.18268352 0.1837613

In [ ]:
dataset['valeur_conf']=  matrice_de_confiance # ajouter une colonne qui s'appelle 'valeur_conf ' qui contient la valeur de confinace de chaque tuple(user, item)
dataset.head()

,Unnamed: 0,user_id,item_id,rating,text,fans,average_stars,friends,vote_funny,useful,...,Guns & Ammo,Telecommunications,Printing Services,Hair Removal,Vape Shops,Watch Repair,Party & Event Planning,Specialty Schools,valeur_conf,sentiment
0,0,0,575,3,"['we', 'dont', 'actually', 'dress', 'like', 'l...",0.008016,0.805851,0.012982,0.001776,0.002434,...,0,0,0,0,0,0,0,0,1.0,0.589736
1,1,0,3529,3,"['spent', 'three', 'hours', 'walking', 'around...",0.008016,0.805851,0.012982,0.001776,0.002434,...,0,0,0,0,0,0,0,0,1.0,0.031666
2,2,0,2126,4,"['went', 'goleta', 'location', 'hollister', 'a...",0.008016,0.805851,0.012982,0.001776,0.002434,...,0,0,0,0,0,0,0,0,1.0,0.602304
3,3,0,2764,5,"['im', 'giving', 'fifi', 'mahoneys', 'five', '...",0.008016,0.805851,0.012982,0.001776,0.002434,...,0,0,0,0,0,0,0,0,1.0,0.600164
4,4,0,7990,4,"['nice', 'selection', 'though', 'much', 'vinta...",0.008016,0.805851,0.012982,0.001776,0.002434,...,0,0,0,0,0,0,0,0,1.0,0.942426


In [ ]:
dataset.to_csv(r'/content/drive/MyDrive/yelp_normalized_dataset(55738).csv', index=False)

In [ ]:
# dataset yelp beauty and spa
dataset.to_csv('/content/drive/MyDrive/Yelp_normalized_Beauty&Spas_Dataset(76449).csv',index=False)

# **Split data**
*pour le decoupage de dataset pour training, validation et test*

## **Alpha coefficient**
*pour ajuter la valeur de rating original*

In [ ]:
def updatescore(y , sentiment ,alpha):
  y=y*(1-alpha)
  sentiment = sentiment*5 # car le sentiment est dans l'intervall [0,1]
  sentiment = sentiment*(alpha)
  return (sentiment+y)/5

In [ ]:
dataset['updaterating']=updatescore(dataset['rating'], dataset['SAFR'] , 0.3)

## **découpage de données entre training, validation et test**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset

# Split into features (X) and labels (y)
X = dataset.drop(columns=['updaterating']) # si vous utilisez le sentiment remplacez 'rating' par 'updaterating'
y = dataset['updaterating']

# Split into training set and remaining data
X_train, remaining_X, y_train, remaining_y = train_test_split(X, y, test_size=0.4, random_state=42)

# Identify unique users and items in training set
unique_users_train = X_train['user_id'].unique()
unique_items_train = X_train['item_id'].unique()

# Filter validation and test data
validation_data = remaining_X[remaining_X['user_id'].isin(unique_users_train) & remaining_X['item_id'].isin(unique_items_train)]
y_val = remaining_y[remaining_X['user_id'].isin(unique_users_train) & remaining_X['item_id'].isin(unique_items_train)]
test_data = remaining_X[~remaining_X.index.isin(validation_data.index)]
y_test = remaining_y[~remaining_X.index.isin(validation_data.index)]

# Split validation and test data
X_val, X_test, y_val, y_test = train_test_split(validation_data, y_val, test_size=0.5, random_state=42)

# Print sizes of training, validation, and test sets
print("Training X shape: ", X_train.shape)
print("Training y shape: ", y_train.shape)
print("Validation X shape: ", X_val.shape)
print("Validation y shape: ", y_val.shape)
print("Test X shape: ", X_test.shape)
print("Test y shape: ", y_test.shape)


Training X shape:  (33442, 125)
Training y shape:  (33442,)
Validation X shape:  (10328, 125)
Validation y shape:  (10328,)
Test X shape:  (10328, 125)
Test y shape:  (10328,)


In [ ]:
#  getnum_user_item fonction permet de recupere le nombre de user and item dans notre dataset
def getnum_user_item(X_train):
    num_users = int(max(X_train.user_id.max()+1,X_test.user_id.max()+1))
    num_items = int(max(X_train.item_id.max()+1,X_test.item_id.max()+1))
    return num_users , num_items

num_users , num_items = getnum_user_item(X_train)

In [ ]:
train_userID = X_train['user_id']
train_itemID = X_train['item_id']
val_userID = X_val['user_id']
val_itemID = X_val['item_id']

### **dataset yelp shopping**

In [ ]:
train_userDATA = X_train[['fans','average_stars','friends','vote_funny','useful','vote_cool','hot','more',
        'profile','cute','list','note','plain',
        'cool','funny','writer','photos']]

train_itemDATA = X_train[['Shopping','Fashion','Food','Local Services','Home Services','Grocery',
                          'Drugstores','Department Stores','Flowers & Gifts','Beauty & Spas','Home Decor','Health & Medical',
                          'Furniture Stores','Electronics','Sporting Goods','Cosmetics & Beauty Supply','Arts & Crafts','Accessories','Jewelry',
                          'Arts & Entertainment','Event Planning & Services','Nurseries & Gardening','Appliances',
                          'Shoe Stores','Discount Store','Active Life','Music & Video','Mags','Books','Hardware Stores',
                          'Eyewear & Opticians','Florists','Specialty Food','Restaurants','Mobile Phones'
                          'Convenience Stores','Optometrists','Used','Vintage & Consignment','Gift Shops',
                          'IT Services & Computer Repair','Art Galleries','Building Supplies','Thrift Stores',
                          'Automotive','Mattresses','Wine & Spirits','Beer','Hair Salons','Professional Services',
                          'Kitchen & Bath','Bridal','Sports Wear','Tobacco Shops','Antiques','Pharmacy','Toy Stores',
                          'Bookstores','Hobby Shops','Bikes','Nightlife','Shopping Centers','Education','Appliances & Repair',
                          'Local Flavor','Photography Stores & Services','Skin Care','Bakeries','Doctors','Mobile Phone Repair',
                          'Outdoor Gear','Wholesale Stores','Computers','Art Supplies','Bars','Coffee & Tea','Watches','Jewelry Repair',
                          'Mobile Phone Accessories','Electronics Repair','Fruits & Veggies','Office Equipment','Venues & Event Spaces',
                          'Contractors','Wedding Planning','Bike Repair/Maintenance','Ophthalmologists','Music & DVDs','Bike Rentals',
                          'Formal Wear','Interior Design','Baby Gear & Furniture','Landscaping','Guns & Ammo',
                          'Telecommunications','Printing Services','Hair Removal','Vape Shops','Watch Repair',
                          'Party & Event Planning','Specialty Schools']]
val_userDATA = X_val[['fans','average_stars','friends','vote_funny','useful','vote_cool','hot','more',
        'profile','cute','list','note','plain',
        'cool','funny','writer','photos']]

val_itemDATA = X_val[['Shopping','Fashion','Food','Local Services','Home Services','Grocery',
                          'Drugstores','Department Stores','Flowers & Gifts','Beauty & Spas','Home Decor','Health & Medical',
                          'Furniture Stores','Electronics','Sporting Goods','Cosmetics & Beauty Supply','Arts & Crafts','Accessories','Jewelry',
                          'Arts & Entertainment','Event Planning & Services','Nurseries & Gardening','Appliances',
                          'Shoe Stores','Discount Store','Active Life','Music & Video','Mags','Books','Hardware Stores',
                          'Eyewear & Opticians','Florists','Specialty Food','Restaurants','Mobile Phones'
                          'Convenience Stores','Optometrists','Used','Vintage & Consignment','Gift Shops',
                          'IT Services & Computer Repair','Art Galleries','Building Supplies','Thrift Stores',
                          'Automotive','Mattresses','Wine & Spirits','Beer','Hair Salons','Professional Services',
                          'Kitchen & Bath','Bridal','Sports Wear','Tobacco Shops','Antiques','Pharmacy','Toy Stores',
                          'Bookstores','Hobby Shops','Bikes','Nightlife','Shopping Centers','Education','Appliances & Repair',
                          'Local Flavor','Photography Stores & Services','Skin Care','Bakeries','Doctors','Mobile Phone Repair',
                          'Outdoor Gear','Wholesale Stores','Computers','Art Supplies','Bars','Coffee & Tea','Watches','Jewelry Repair',
                          'Mobile Phone Accessories','Electronics Repair','Fruits & Veggies','Office Equipment','Venues & Event Spaces',
                          'Contractors','Wedding Planning','Bike Repair/Maintenance','Ophthalmologists','Music & DVDs','Bike Rentals',
                          'Formal Wear','Interior Design','Baby Gear & Furniture','Landscaping','Guns & Ammo',
                          'Telecommunications','Printing Services','Hair Removal','Vape Shops','Watch Repair',
                          'Party & Event Planning','Specialty Schools']]

### **dataset Yelp-beauty and spa**

In [ ]:
train_userDATA = X_train[['fans','average_stars','friends','vote_funny','vote_cool','hot','more',
                          'profile','cute','list','note','plain','writer','photos']]

train_itemDATA = X_train[['Beauty & Spas', 'Nail Salons',	'Hair Salons',	'Hair Removal',	'Waxing',	'Skin Care',	'Health & Medical',	'Massage',	'Day Spas',
                          'Shopping',	'Eyelash Service',	'Cosmetics & Beauty Supply',	'Barbers',	'Hair Stylists',	'Massage Therapy',	'Makeup Artists']]

#Validation
val_userDATA = X_val[['fans','average_stars','friends','vote_funny','vote_cool','hot','more',
                          'profile','cute','list','note','plain','writer','photos']]
val_itemDATA = X_val[['Beauty & Spas', 'Nail Salons',	'Hair Salons',	'Hair Removal',	'Waxing',	'Skin Care',	'Health & Medical',	'Massage',	'Day Spas',
                          'Shopping',	'Eyelash Service',	'Cosmetics & Beauty Supply',	'Barbers',	'Hair Stylists',	'Massage Therapy',	'Makeup Artists']]


KeyError: ignored

# **Neural Hybrid Filtring model**

In [ ]:
#  cree le model NHF
def build_NHF_model(emb_size_hmlp, emb_size_gmf, predictive_factors, num_layers):

    # définition des input pour les deux model gmf et mlp
    input_userID = Input(shape = [1], name = 'user_ID')
    input_itemID = Input(shape = [1], name = 'item_ID')

    input_userDATA = Input(shape = [17], name = 'user_data')
    input_itemDATA = Input(shape = [100], name = 'item_data')
    # input_userDATA = Input(shape = [14], name = 'user_data')
    # input_itemDATA = Input(shape = [16], name = 'item_data')

    #GMF part
    user_latent_factors_GMF = emb_size_gmf
    item_latent_factors_GMF = emb_size_gmf

    user_emb_GMF = Embedding(num_users, user_latent_factors_GMF, embeddings_initializer="uniform",name = 'user_emb_GMF')(input_userID)
    item_emb_GMF = Embedding(num_items, item_latent_factors_GMF, embeddings_initializer="uniform", name = 'item_emb_GMF')(input_itemID)

    flat_u_GMF = Flatten()(user_emb_GMF)
    flat_i_GMF = Flatten()(item_emb_GMF)

    mul_layer = Multiply()([flat_u_GMF, flat_i_GMF]) # user embedding and item embedding

    #HybMLP part
    user_latent_factors_hMLP = emb_size_hmlp
    item_latent_factors_hMLP = emb_size_hmlp

    user_emb_hMLP = Embedding(num_users, user_latent_factors_hMLP, embeddings_initializer="uniform", name= 'user_emb_hMLP')(input_userID)
    item_emb_hMLP = Embedding(num_items, item_latent_factors_hMLP, embeddings_initializer="uniform", name= 'item_emb_hMLP')(input_itemID)

    flat_u_hMLP = Flatten()(user_emb_hMLP)
    flat_i_hMLP = Flatten()(item_emb_hMLP)

    concat_hMLP = concatenate([flat_u_hMLP, flat_i_hMLP, input_userDATA, input_itemDATA])
    layer = concat_hMLP
    for l in range(num_layers,0,-1):
        layer = Dense(predictive_factors*(2**(l-1)), activation='tanh' , name= 'layer%d' %(num_layers-l+1))(layer)
        layer = tf.keras.layers.BatchNormalization()(layer)
# kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-4, l2=1e-4)

    #NeuHybMF part :  concatener le model gmf et mlp
    concat_NeuhMF = concatenate([mul_layer,layer])

    out = Dense(1, activation='sigmoid', name='output')(concat_NeuhMF)
    #  le model final de filtrage :

    # definir les inputs/outputs
    NHF_model = Model([input_userID,input_userDATA, input_itemID ,input_itemDATA], out)

    return NHF_model

In [ ]:
from keras import metrics
rmse = metrics.RootMeanSquaredError()
precision = metrics.Precision()

## **training**

In [ ]:
from tensorflow.keras.metrics import RootMeanSquaredError

# callback = tf.keras.callbacks.EarlyStopping(patience = 5, restore_best_weights=True)

num_emb_mlp=8
num_emb_gmf=8
PF=16
MLPLAYERS= 5
batch = 32
epochs= 10
f_model = build_NHF_model(num_emb_mlp, num_emb_gmf, PF, MLPLAYERS )

In [ ]:
f_model.compile( tf.keras.optimizers.RMSprop(8e-5),
                loss="mean_absolute_error",
                metrics=[rmse,'mae'])

In [ ]:
# y_train = y_train/5
# y_val = y_val/5 # dans le cas ou vous n'avez pas utilisez le sentiment

In [ ]:
history = f_model.fit(x=[train_userID,train_userDATA,train_itemID,train_itemDATA], y=y_train, batch_size = batch, epochs = epochs ,validation_data=([val_userID,val_userDATA,val_itemID,val_itemDATA] , y_val))#,callbacks =[callback])

Epoch 1/10
1046/1046 [==============================] - 28s 17ms/step - loss: 0.2635 - root_mean_squared_error: 0.3235 - mae: 0.2635 - val_loss: 0.2131 - val_root_mean_squared_error: 0.2699 - val_mae: 0.2131
Epoch 2/10
1046/1046 [==============================] - 11s 10ms/step - loss: 0.2048 - root_mean_squared_error: 0.2651 - mae: 0.2048 - val_loss: 0.1766 - val_root_mean_squared_error: 0.2383 - val_mae: 0.1766
Epoch 3/10
1046/1046 [==============================] - 9s 9ms/step - loss: 0.1668 - root_mean_squared_error: 0.2146 - mae: 0.1668 - val_loss: 0.1497 - val_root_mean_squared_error: 0.1957 - val_mae: 0.1497
Epoch 4/10
1046/1046 [==============================] - 11s 11ms/step - loss: 0.1510 - root_mean_squared_error: 0.1998 - mae: 0.1510 - val_loss: 0.1466 - val_root_mean_squared_error: 0.1974 - val_mae: 0.1466
Epoch 5/10
1046/1046 [==============================] - 10s 10ms/step - loss: 0.1480 - root_mean_squared_error: 0.1999 - mae: 0.1480 - val_loss: 0.1460 - val_root_mean_sq

In [ ]:
from tensorflow.python.ops.array_ops import placeholder_with_default_eager_fallback
import matplotlib.pyplot as plt
loss_curve = history.history["loss"]
rmse_curve = history.history["root_mean_squared_error"]
mae_curve  = history.history["mae"]

loss_val   = history.history["val_loss"]
rmse_val   = history.history["val_root_mean_squared_error"]
mae_val    = history.history["val_mae"]


# ploter loss function
def ploter (title , curve , validation):
  plt.plot(curve , label = "train")
  plt.plot(validation , label = "validation")
  plt.legend(loc='upper left')
  # plt.ylim(0,1)
  plt.title(title)
  plt.show()

ploter ('loss' , loss_curve , loss_val)
ploter ('rmse' , rmse_curve , rmse_val)
ploter ('mae' , mae_curve , mae_val)

## **test**

In [ ]:
f_model = tf.keras.models.load_model('/content/drive/MyDrive/testNHFmodel.h5')

### **data set Yelp-shopping**

In [ ]:
test_userDATA = X_test[['fans','average_stars','friends','vote_funny','useful','vote_cool','hot','more',
        'profile','cute','list','note','plain',
        'cool','funny','writer','photos']]

test_itemDATA = X_test[['Shopping','Fashion','Food','Local Services','Home Services','Grocery',
                        'Drugstores','Department Stores','Flowers & Gifts','Beauty & Spas','Home Decor','Health & Medical',
                        'Furniture Stores','Electronics','Sporting Goods','Cosmetics & Beauty Supply','Arts & Crafts','Accessories','Jewelry',
                        'Arts & Entertainment','Event Planning & Services','Nurseries & Gardening','Appliances',
                        'Shoe Stores','Discount Store','Active Life','Music & Video','Mags','Books','Hardware Stores',
                        'Eyewear & Opticians','Florists','Specialty Food','Restaurants','Mobile Phones'
                        'Convenience Stores','Optometrists','Used','Vintage & Consignment','Gift Shops',
                        'IT Services & Computer Repair','Art Galleries','Building Supplies','Thrift Stores',
                        'Automotive','Mattresses','Wine & Spirits','Beer','Hair Salons','Professional Services',
                        'Kitchen & Bath','Bridal','Sports Wear','Tobacco Shops','Antiques','Pharmacy','Toy Stores',
                        'Bookstores','Hobby Shops','Bikes','Nightlife','Shopping Centers','Education','Appliances & Repair',
                        'Local Flavor','Photography Stores & Services','Skin Care','Bakeries','Doctors','Mobile Phone Repair',
                        'Outdoor Gear','Wholesale Stores','Computers','Art Supplies','Bars','Coffee & Tea','Watches','Jewelry Repair',
                        'Mobile Phone Accessories','Electronics Repair','Fruits & Veggies','Office Equipment','Venues & Event Spaces',
                        'Contractors','Wedding Planning','Bike Repair/Maintenance','Ophthalmologists','Music & DVDs','Bike Rentals',
                        'Formal Wear','Interior Design','Baby Gear & Furniture','Landscaping','Guns & Ammo',
                        'Telecommunications','Printing Services','Hair Removal','Vape Shops','Watch Repair',
                        'Party & Event Planning','Specialty Schools']]

In [ ]:
test_userID = X_test['user_id']
test_itemID = X_test['item_id']

### **data set Yelp-beauty and spa**

In [ ]:
test_userDATA = X_test[['fans','average_stars','friends','vote_funny','vote_cool','hot','more',
                          'profile','cute','list','note','plain','writer','photos']]

test_itemDATA = X_test[['Beauty & Spas', 'Nail Salons',	'Hair Salons',	'Hair Removal',	'Waxing',	'Skin Care',	'Health & Medical',	'Massage',	'Day Spas',
                          'Shopping',	'Eyelash Service',	'Cosmetics & Beauty Supply',	'Barbers',	'Hair Stylists',	'Massage Therapy',	'Makeup Artists']]

test_userID = X_test['user_id']
test_itemID = X_test['item_id']

In [ ]:
# y_test= y_test/5

### ***evaluation***

In [ ]:
scores = f_model.evaluate([test_userID,test_userDATA,test_itemID,test_itemDATA],y_test,verbose=1)

In [ ]:
scores

In [ ]:
f_model.save('/content/drive/MyDrive/concat_modelCASAREC', save_format='tf')

In [ ]:
loaded_model_SA = tf.keras.models.load_model('/content/drive/MyDrive/concat_modelCASAREC')

In [ ]:
loaded_model_SA.evaluate([test_userID,test_userDATA,test_itemID,test_itemDATA],y_test)

323/323 [==============================] - 2s 4ms/step - loss: 0.1436 - root_mean_squared_error: 0.1960 - mae: 0.1436


[0.14358997344970703, 0.1960182785987854, 0.14358997344970703]

In [ ]:
rec = loaded_model_SA.predict([test_userID,test_userDATA,test_itemID,test_itemDATA])

323/323 [==============================] - 2s 5ms/step


In [ ]:
res = rec*5
X_test["predicted_rating"]=res
X_test['rating']=y_test*5

In [ ]:
X_test[['user_id','item_id',"rating",'predicted_rating']]

,user_id,item_id,rating,predicted_rating
51752,2231,6094,4.866717,4.556844
23201,912,293,4.888474,4.261788
17937,689,278,1.341289,3.970190
13649,493,8329,4.969877,4.716874
52357,2268,203,4.880066,4.611801
...,...,...,...,...
31650,1275,2660,3.413365,4.659719
26750,1078,3475,1.500549,3.574721
9666,340,516,3.505952,3.862087
30900,1245,7342,4.267752,4.170387
